In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
from pathlib import Path
import sys, os
utils_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm/src/utils")
sys.path.append(str(utils_path))

In [53]:
import pandas as pd
import numpy as np

from pdf2image import convert_from_path
from PIL import Image

from unstructured.partition.pdf import partition_pdf

from langchain_core.documents import Document
# from langchain.vectorstores import FAISS

from utils import generate_random_name

## Define Paths

In [54]:
base_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm")
artifacts_path = os.path.join(base_path, "artifacts")
poppler_path = os.path.join(artifacts_path, "libraries/Release-23.08.0-0/poppler-23.08.0/Library/bin")
supp_apps_pdfs_dir_path = os.path.join(artifacts_path, "applications/pdfs")
supp_apps_pngs_dir_path = os.path.join(artifacts_path, "applications/pngs")
supp_apps_chunked_pngs_dir_path = os.path.join(artifacts_path, "applications/chunked_pngs")

## PDF to PNG

In [55]:
example_pdf_path = os.path.join(supp_apps_pdfs_dir_path, "amwins_app.pdf")

In [56]:
example_pdf_pages = convert_from_path(pdf_path=example_pdf_path, poppler_path=poppler_path)
example_pdf_name = os.path.splitext(os.path.basename(example_pdf_path))[0]
c=1
for page in example_pdf_pages:
    os.makedirs(supp_apps_pngs_dir_path, exist_ok=True)
    img_name = f"{example_pdf_name}_{c}.png"
    page.save(os.path.join(supp_apps_pngs_dir_path, img_name), "PNG")
    c+=1

## Unstructured IO tests

In [57]:
elements = partition_pdf(
    filename=example_pdf_path,
    strategy="hi_res",
)

In [58]:
documents = []
for element in elements:
    if len(element.text)<10: continue
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

# Chunk images and update documents with chunked img paths
c=1
for document in documents:
    chunk_page_no = document.metadata['page_number']
    matched_page_img_name = f"{example_pdf_name}_{chunk_page_no}.png"

    matched_page_img_path = os.path.join(supp_apps_pngs_dir_path, matched_page_img_name)
    matched_page_img = Image.open(matched_page_img_path)

    chunked_coords = document.metadata['coordinates']['points']
    chunked_region = (chunked_coords[0][0], chunked_coords[0][1], chunked_coords[2][0], chunked_coords[2][1])
    chunked_img =  matched_page_img.crop(chunked_region)

    os.makedirs(supp_apps_chunked_pngs_dir_path, exist_ok=True)
    chunked_img_name = f"{example_pdf_name}_{c}.png"
    chunked_img_path = os.path.join(supp_apps_chunked_pngs_dir_path, chunked_img_name)
    chunked_img.save(chunked_img_path)
    document.metadata['chunked_img_path'] = chunked_img_path

    c+=1
